<a href="https://colab.research.google.com/github/zr-morris/CrewAI-Colab/blob/main/CrewAI-Accrued-Expense-Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install crewai crewai[tools]  --q

In [8]:
import os
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
import chardet

# Assuming the placeholder imports match your project's structure
from crewai import Agent, Task, Crew
from crewai_tools import DirectoryReadTool, PDFSearchTool, CSVSearchTool, DOCXSearchTool, TXTSearchTool

def on_submit(b):
    api_key = api_key_widget.value
    directory_path = directory_path_widget.value
    task_description = task_description_widget.value.strip()

    if not api_key or not directory_path or not task_description:
        with output:
            clear_output()
            print("Please fill out all fields.")
        return

    try:
        os.environ["OPENAI_API_KEY"] = api_key

        # Instantiate tools with the user-provided directory path
        docs_tool = DirectoryReadTool(directory=directory_path)
        pdf_rag_tool = PDFSearchTool()
        csv_rag_tool = CSVSearchTool()
        docx_rag_tool = DOCXSearchTool()
        txt_rag_tool = TXTSearchTool()

        # Create agents with the instantiated tools
        accrued_expense_agent = Agent(
            role='Accrued Expense Agent',
            goal="""Perform the substantive audit test for accrued expenses\
            You are aware of how accrual accounting works and you know how to think about pro-rata expenses, and appropriate accruals.\
            For contracts or agreements for services rendered over time, you first calculate the portion of the contract or agreement that should be accrued.\
            For point in time expenses you ensure there is an invoice that supports the accrued amount, you do not need a contract or agreement to support these point in time expenses accruals.""",
            backstory="""You are an expert Certified Professional Accountant performing an audit of expenses for your client.""",
            allow_delegation=False,
            verbose=True,
            max_iter=10,
            tools=[docs_tool, pdf_rag_tool, csv_rag_tool, docx_rag_tool, txt_rag_tool]
        )

        # Dynamically create a task with the user input
        dynamic_task = Task(
            description=task_description,
            expected_output="""A professionally formatted analytical audit report that shows the results of the accrued expenses testwork you have performed.\
            Show the values audited, along with the values of any potential misstatements.\
            Provide explanations about test failures that result in a potential factual misstatement.""",
            agent=accrued_expense_agent
        )

        # Assemble and execute the task with Crew
        crew = Crew(
            agents=[accrued_expense_agent],
            tasks=[dynamic_task],
            verbose=True,
            output_file='accrued_expense_report.md'
        )
        crew.kickoff()

        # Display the content of the generated markdown file
        display_markdown(dynamic_task.output_file)

    except Exception as e:
        with output:
            clear_output()
            print(f"Error: {str(e)}")

def display_markdown(file_path):
    """Load and display the markdown file content"""
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        encoding = chardet.detect(raw_data)['encoding']
        if encoding:
            content = raw_data.decode(encoding)
            with output:
                clear_output()
                display(Markdown(content))
        else:
            with output:
                clear_output()
                print("Failed to detect file encoding.")

# Create label and input widgets
api_key_label = widgets.Label('OpenAI API Key:')
api_key_widget = widgets.Text(layout=widgets.Layout(width='90%'))

directory_path_label = widgets.Label('Directory Path:')
directory_path_widget = widgets.Text(layout=widgets.Layout(width='90%'))

task_description_label = widgets.Label('Task Description:')
task_description_widget = widgets.Textarea(layout=widgets.Layout(width='90%', height='200px'))

submit_button = widgets.Button(description="Submit")
output = widgets.Output()

submit_button.on_click(on_submit)

# Display widgets with labels
display(api_key_label, api_key_widget)
display(directory_path_label, directory_path_widget)
display(task_description_label, task_description_widget)
display(submit_button, output)


Label(value='OpenAI API Key:')

Text(value='', layout=Layout(width='90%'))

Label(value='Directory Path:')

Text(value='', layout=Layout(width='90%'))

Label(value='Task Description:')

Textarea(value='', layout=Layout(height='200px', width='90%'))

Button(description='Submit', style=ButtonStyle())

Output()

 [DEBUG]: == Working Agent: Accrued Expense Agent
 [INFO]: == Starting Task: Perform accrued expense substantive testing over ALL of the transactions in the markdown table below.

All supporting documents are found in {file path}: '/content/accrued_expense_support'. All documents required for the test are in this folder. The listing of transacations is found in {file path}:'/content/accrued_expense_support/ExtractionReport_1682652900174-problem.csv'

Ensure accrued expenses conform with the individual agreements underlying the expense. The pro-rated portion of the agreements should be accrued based upon the 'effective date' of the contract and the total premium amount if agreement start date is not on January 1st. If the 'Value to Test' amount doesn't equal the correct pro-rated amount given the stated agreement start date, there is a misstatement.

Assess whether the accruals adhere to accrual accounting practices under GAAP, expenses should be accrued as they are incurred, and not al